#### URL去重原理
1. Scrapy自带URL去重Middleware:'dupefilters.py':
    * class BaseDupeFilter(object):默认过滤器  
        |- def request_seen(self, request):  
        * request_seen()在scheduler.py(调度器,调度request的分发)中的def enqueue_request(self, request)被使用
        ```
        def enqueue_request(self, request):
            if not request.dont_filter and self.df.request_seen(request):
                self.df.log(request, self.spider)
                return False
        ```
        * dont_filter参数可以在Request中设置,在enqueue_request()被读取

#### spider middleware
1. 允许在Spider和Engine之间做一些处理,同样需要在setting.py里配置SPIDER_MIDDLEWARE
2. SPIDER_MIDDLEWARE和DOWNLOADER_MIDDLEWARE都是被MiddlewareManager()类所控制的,而MiddlewareManager里定义了from_crawler(),所以说不管SPIDER_MIDDLEWARE还是DOWNLOADER_MIDDLEWARE,都可以去重写from_crawler(),比如可以在from_crawler()中定义某个信号量触发时实现的方法
```
class ArticlespiderSpiderMiddleware(object):
    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s
        
    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)
```
<img src='1.png' width=800px>
3. 和DOWNLOADER_MIDDLEWARE一样,SPIDER_MIDDLEWARE有着默认的方法
    * process_spider_input(response, spider)
        * 该方法如果被重写,会在**⑥**被执行
    * process_spider_output(response, result, spider)
        * 该方法如果被重写,会在**⑦**被执行
    * process_spider_exception(response, exception, spider)
    * process_start_requests(start_requests, spider)
        * 该方法如果被重写,会在**①**被执行
    * from_crawler(cls, crawler)